#### 一. 过滤器
1. Filter简介  
    * hbase为scan和get操作提供更加高级的filter  
    * Filter可以根据qualifier,列簇,版本等条件对行进行过滤.减少返回的结果数量, 降低网络传输压力  
    * 过滤操作在RegionServer上执行
2. SingleColumnValueFilter: 实现列簇/列上的where语义  
3. RowFilter, 实现row key的正则表达式过滤
4. FamilyFilter, 
5. QualifierFilter, 
6. FilterList, 实现查询条件and语法  
[各种过滤器](https://www.cnblogs.com/similarface/p/5805973.html)
[布隆过滤器](https://en.wikipedia.org/wiki/Bloom_filter)
[Hbase架构](https://mapr.com/blog/in-depth-look-hbase-architecture/#.VdMxvWSqqko)

#### 二. Compact原理
1. Hbase基于`Log-Structed Merge Tree`架构的   
2. 传统的$B^+$树能够执行快速随机读取, 但随机写入因为要调整树形结构, 因此在大数据量下会变的很慢  
   * 为什么$B^+$树的随机读取很快?  
   $B^+$树是多路查找树, 所以要随机查找一个数值, 将节点调入内存的次数只和树的高度有关. 又因为$B^+$树的每个节点包含很多值, 所以节点调入内存时产生的磁盘IO是顺序IO, 又加快了节点调入内存的速度
3. LSM tree如何快速执行随机写?  
   * LSM tree首先在内存中维护一个有序的树形结构,类似$B^+$树,称作MemStore. 随机写入的数据会在MemStore中调整树型后插入这个有序树中.  
   * 为了保证数据的可靠性, 在写入Memstore前会先以WAL(Write-Ahead-Log)的方式备份写日志  
   * 当MemStore的数据量到达一定条件后, 会被刷新到磁盘上形成一个新文件"HFile". 因为这种写磁盘是顺序写, 没有旧文件被读取修改, 因此执行很快  
   * "HFile"在LSM tree架构里又称sstable, 意为相似的有序文件.当执行随机读取时, 首先在MemStore中查找是否存在这个key, 是的话直接返回, 不是的话查找每个HFile中的记录. 因为HFile中的记录和Memstore一样也是有序的树形结构, 所以查找一个HFile的时间复杂度 $O(logN)$ , 所有的HFile查找时间复杂度 $k*O(logN)$. 因此, 如果HFIle的数量很多, 会使得查找速度变慢, 需要Compact操作 
   * LSM tree主要是对写优化  
 综上, 因为随机写入是分批量的顺序写入磁盘, 所以速度很快
4. Bloom过滤器, 减少HFile的筛选  
  为了减少随机读取时需要查找的HFile数量, 可使用Bloom过滤器先行判断带查找的key是否在这个HFile里. 如果Bloom过滤器返回False, 则key一定不在HFile里, 如果返回True, 仍有一定概率不在这个HFile里
5. Compact  
    * Minor compact  
    将一些小的,相邻的HFile合并成一个大的HFile. 此过程不会处理过期或已删除的kv对  
    * Major compact  
    把一个Region下的所有HFile合并成一个大的HFile. 此过程会清理掉3类无意义的数据: 被删除的数据, TTL过期数据, 版本号超过限制的数据  
    Major compact耗时很长, 通常关闭自动major compact, 改为业务低峰期时手动处理Major Compact

#### 三. Bloom过滤器  
1. 上面的LSM tree架构中提到, 随机读时使用Bloom过滤器减少查找的HFile数量, 在Hbase就是使用Bloom过滤器判断该rowkey是否可能在该HFile中    
2. 算法描述  
    * Bloom过滤器用于判断一个元素是否在一个集合内,是一种概率数据结构  
    它返回"可能在集合内"和"绝不可能在集合内"2个结果. 集合中的元素越多, Bloom误判成可能在集合中的概率就越大
    * Bloom过滤器定义k个hash函数, 和一个长度为m的bit数组(bitmap,java中用BitSet表示).   
    数组被初始化为全0数组, 每个哈希函数会把集合中的元素用射到数组的一个位置, 并把该位置的数组值置1.   
    测试一个元素是否在集合中, 用预先定义的k个hash函数依次哈希该值, 得到k个位置, 查看数组中这k个位置的值是否全部是1, 有一个为0则返回"一定不在集合中"<img src="img/bloomfilter3.png">m的选择决定了Bloom过滤器误判概率的大小  
    * k个hash函数应该选择互相之间少有重叠的, 比如"2次方hash,3次方hash"  
3. 概率解释  
    * 误判成"可能存在"的概率  
    假设hash函数会把一个值等概率的映射到数组的m个位置, 则依次hash后, 该位置的数组没有被置为1的概率为$$1-\frac{1}{m}$$  
    k个不相关的hash函数均未把该位置置1的概率为$${(1-\frac{1}{m})}^k$$
    插入n个元素后, 该位置仍为0的概率$${(1-\frac{1}{m})}^{kn}$$
    即该位置为1的概率$$1-{(1-\frac{1}{m})}^{kn}$$
    * k的选择  
    k只由误判的概率决定, 有$k=-{log_2}P$

#### 四. zookeeper中的meta表
1. meta表的作用
    * 我们知道HBase的表是会分割为多个Region的，不同Region分布到不同RegionServer上。  
    * Region 是 HBase中分布式存储和负载均衡的最小单元。  
    所以当我们从客户端读取，写入数据的时候，我们就需要知道数据的Rowkey是在哪个Region以及我们需要的Region是在哪个RegionServer上。而这正是HBase Meta表所记录的信息。
2. meta表也是一张Hbase的表
    * meta表也是一张Hbase表, 有rowkey, 列族, 列
    * hbase的所有查询操作, 都要先访问meta表, 找到自己所查记录的表和rowkey在哪个Region上, 再到那个Region上执行memstore或hfile的查找
3. meta表的rowkey
    * meta表的rowkey组成结构: table名 + region的start key + 时间戳
    * 上述3者的MD5值也是HBase在HDFS上的region名
4. meta表的列族和列
    * 最主要的Family：info
    * info里面包含三个列：regioninfo, server, serverstartcode。
    其中regioninfo就是Region的详细信息，包括StartKey, EndKey 以及每个Family的信息等。server存储的就是管理这个Region的RegionServer的地址。所以当Region被拆分、合并或者重新分配的时候，都需要来修改这张表的内容。
    
5. Hbase数据查询过程: 
    1. 客户端连接zookeeper, 查找meta表在哪个机器上
    2. 查询meta表, 根据每个region的`startKey`和`endKey`找到region所在的server
    3. 到指定Server上查找该region
    4. meta标的信息会被缓存起来, 便于下次查找

#### 五. HBase的master-slave架构
* HMaster
    * 每个HregionServer都与HMaster保持心跳通信.
    * HMaster负责给HRegionServer分配HRegion
    * 当一台HRegionServer宕机, HMaster会把这个RegionServer的Region标记为未分配, 然后再把他们分配到其他HRegionServer中
    * HMaster的功能: HMaster主要负责`Table`和`Region`的管理工作   
        * 管理用户对Table级别的增删改查
        * 负责HRegionServer的负载均衡, 调整Region在每台机器的分布
        * 在Region分片后, 负责新的Region分配
        * 在HRegion Server停机后, 负责失效的HRegionServer上的Regions迁移
* HRegionServer
    * 用户通过访问HRegionServer获取这些数据
    * 一台机器上面一般只运行一个HRegionServer
    * 一个HRegionServer上面有多个HRegion，一个HRegion 也只会被
    * 一个HRegionServer维护